In [1]:
from random import randint

def dice(num):
    """ num 個のサイコロの合計を返す """
    total = 0
    for _ in range(num):
        # randint は実引数で指定した範囲のランダムな整数を返す
        total += randint(1, 6)
    return total

def countall(num):
    """ num 個のサイコロを振った場合に、すべての出目が出るまでの回数を返す"""
    
    # 出現した出目を記録する set
    s = set()
    # 出目の種類を計算する。最小値が num で 最大値が num * 6
    left = num * 5 + 1
    # サイコロを振った数を数える変数を初期化する
    count = 0
    # 全ての出目が出るまで繰り返す
    while left > 0:
        # サイコロを振る
        dicenum = dice(num)
        # 振った回数を増やす
        count += 1
        # s の要素に登録されているかを判定する
        if dicenum not in s:
            # 登録されていなければ、left を 1 減らし s に出目を登録する
            left -= 1
            s.add(dicenum)
    return count

# 2 ~ 10 個のそれぞれの場合の回数を表示する
for i in range(2, 11):
    print(i, countall(i))

2 45
3 333
4 3746
5 2333
6 99256
7 833719
8 4610084
9 10782768
10 51201303


In [2]:
from ai import ai_match, ai2, ai11s

param_ai11s = {
    "score_201": 2,
    "score_102": 1,
    "score_012": -1
}
ai_match(ai=[ai11s, ai2], params=[param_ai11s, {}])

ai11s VS ai2
count     win    lose    draw
o        9879       0     121
x        8767     236     997
total   18646     236    1118

ratio     win    lose    draw
o       98.8%    0.0%    1.2%
x       87.7%    2.4%   10.0%
total   93.2%    1.2%    5.6%



In [3]:
from marubatsu import Marubatsu, Markpat
from ai import ai_by_score

def ai11s(mb, score_201=2, score_102=1, score_012=-1, debug=False):
    def eval_func(mb):      
        # 真ん中のマスに着手している場合は、評価値として 300 を返す
        if mb.last_move == (1, 1):
            return 300
    
        # 自分が勝利している場合は、評価値として 200 を返す
        if mb.status == mb.last_turn:
            return 200

        markpats = mb.count_markpats()
        if debug:
            pprint(markpats)
        # 相手が勝利できる場合は評価値として -100 を返す
        if markpats[Markpat(last_turn=0, turn=2, empty=1)] > 0:
            return -100
        # 次の自分の手番で自分が必ず勝利できる場合は評価値として 100 を返す
        elif markpats[Markpat(last_turn=2, turn=0, empty=1)] >= 2:
            return 100

        # 評価値の合計を計算する変数を 0 で初期化する
        score = 0        
        # 次の自分の手番で自分が勝利できる場合は評価値に score_201 を加算する
        if markpats[Markpat(last_turn=2, turn=0, empty=1)] == 1:
            score += score_201
        # 「自 1 敵 0 空 2」1 つあたり score_102 だけ、評価値を加算する
        score += markpats[Markpat(last_turn=1, turn=0, empty=2)] * score_102
        # 「自 0 敵 1 空 2」1 つあたり score_201 だけ、評価値を減算する
        score += markpats[Markpat(last_turn=0, turn=1, empty=2)] * score_012
        
        # 計算した評価値を返す
        return score

    return ai_by_score(mb, eval_func, debug=debug)

In [4]:
mb = Marubatsu()
print(ai11s(mb))

(1, 1)


In [5]:
ai_match(ai=[ai11s, ai2], params=[{}, {}])

ai11s VS ai2
count     win    lose    draw
o        9896       0     104
x        8714     239    1047
total   18610     239    1151

ratio     win    lose    draw
o       99.0%    0.0%    1.0%
x       87.1%    2.4%   10.5%
total   93.0%    1.2%    5.8%



In [9]:
from collections import defaultdict

def ai_match(ai, params=[{}, {}], match_num=10000):
    print(f"{ai[0].__name__} VS {ai[1].__name__}")
    
    mb = Marubatsu()

    # ai[0] VS ai[1] と ai[1] VS a[0] の対戦を match_num 回行い、通算成績を数える
    count_list = [ defaultdict(int), defaultdict(int)]
    for _ in range(match_num):
        count_list[0][mb.play(ai, params, verbose=False)] += 1
        count_list[1][mb.play(ai=ai[::-1], params=params[::-1], verbose=False)] += 1

    # ai[0] から見た通算成績を計算する
    count_list_ai0 = [
        # ai[0] VS ai[1] の場合の、ai[0] から見た通算成績
        { 
            "win": count_list[0][Marubatsu.CIRCLE],
            "lose": count_list[0][Marubatsu.CROSS],
            "draw": count_list[0][Marubatsu.DRAW],
        },
        # ai[1] VS ai[0] の場合の、ai[0] から見た通算成績
        { 
            "win": count_list[1][Marubatsu.CROSS],
            "lose": count_list[1][Marubatsu.CIRCLE],
            "draw": count_list[1][Marubatsu.DRAW],
        },
    ]           

    # 両方の対戦の通算成績の合計を計算する
    count_list_ai0.append({})
    for key in count_list_ai0[0]:
        count_list_ai0[2][key] = count_list_ai0[0][key] + count_list_ai0[1][key]

    # それぞれの比率を計算し、ratio_list に代入する
    ratio_list = [ {}, {}, {} ]
    for i in range(3):
        for key in count_list_ai0[i]:
            ratio_list[i][key] = count_list_ai0[i][key] / sum(count_list_ai0[i].values())
            
    # 各行の先頭に表示する文字列のリスト
    item_text_list = [ Marubatsu.CIRCLE, Marubatsu.CROSS, "total" ]    
    
    # 通算成績の回数と比率の表示
    width = max(len(str(match_num * 2)), 7)
    diff_list = [ ("count", count_list_ai0, f"{width}d"),
                  ("ratio", ratio_list, f"{width}.1%") ]
    for title, data, format in diff_list:
        print(title, end="")
        for key in data[0]:
            print(f" {key:>{width}}", end="")
        print()
        for i in range(3):
            print(f"{item_text_list[i]:5}", end="")
            for value in data[i].values():
                print(f" {value:{format}}", end="")
            print()
        print()

In [10]:
ai_match(ai=[ai11s, ai2])

ai11s VS ai2
count     win    lose    draw
o        9903       0      97
x        8715     240    1045
total   18618     240    1142

ratio     win    lose    draw
o       99.0%    0.0%    1.0%
x       87.2%    2.4%   10.4%
total   93.1%    1.2%    5.7%



In [11]:
from ai import show_progress

show_progress(ai=[ai2, ai11s], winner=Marubatsu.CIRCLE)

TypeError: Marubatsu.play() missing 1 required positional argument: 'params'

In [12]:
def play(self, ai, params=[{}, {}], verbose=True):
    # 〇×ゲームを再起動する
    self.restart()
    # ゲームの決着がついていない間繰り返す
    while self.status == Marubatsu.PLAYING:
        # ゲーム盤の表示
        if verbose:
            print(self)
        # 現在の手番を表す ai のインデックスを計算する
        index = 0 if self.turn == Marubatsu.CIRCLE else 1
        # ai が着手を行うかどうかを判定する
        if ai[index] is not None:
            x, y = ai[index](self, **params[index])
        else:
            # キーボードからの座標の入力
            coord = input("x,y の形式で座標を入力して下さい。exit を入力すると終了します")
            # "exit" が入力されていればメッセージを表示して関数を終了する
            if coord == "exit":
                print("ゲームを終了します")
                return       
            # x 座標と y 座標を要素として持つ list を計算する
            xylist = coord.split(",")
            # xylist の要素の数が 2 ではない場合
            if len(xylist) != 2:
                # エラーメッセージを表示する
                print("x, y の形式ではありません")
                # 残りの while 文のブロックを実行せずに、次の繰り返し処理を行う
                continue
            x, y = xylist
        # (x, y) に着手を行う
        try:
            self.move(int(x), int(y))
        except:
            print("整数の座標を入力して下さい")

    # 決着がついたので、ゲーム盤を表示する
    if verbose:
        print(self)
    return self.status

Marubatsu.play = play

In [13]:
show_progress(ai=[ai2, ai11s], winner=Marubatsu.CIRCLE)

Turn x
...
...
O..

Turn o
...
.X.
o..

Turn x
...
.xO
o..

Turn o
X..
.xo
o..

Turn x
x..
.xo
o.O

Turn o
x..
Xxo
o.o

winner o
x.O
xxo
o.o



In [14]:
def show_progress(ai, winner, params=[{}, {}]):
    mb = Marubatsu()
    while True:
        if mb.play(ai=ai, verbose=False, params=params) == winner:
            records = mb.records
            mb.restart()
            for x, y in records:
                mb.move(x, y)
                print(mb)
            break

In [15]:
show_progress(ai=[ai2, ai11s], winner=Marubatsu.CIRCLE)

Turn x
O..
...
...

Turn o
o..
.X.
...

Turn x
o..
.x.
..O

Turn o
o..
.x.
X.o

Turn x
o.O
.x.
x.o

Turn o
o.o
.x.
xXo

winner o
o.o
.xO
xxo

